In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import pandas as pd
from pathlib import Path
import itertools
import numpy as np
import json
import re
import copy
from scipy.stats import mannwhitneyu

import matplotlib.pyplot as plt
import seaborn as sns
import starbars

In [2]:
from rdkit import RDLogger
RDLogger.DisableLog("rdApp.*")
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
cpath = Path(os.getcwd())
output_dir =  cpath.parent / "output" / "vina"
os.makedirs(output_dir, exist_ok=True)
pdb_dir = cpath.parent / "data" / "CrossDocked2020"
dataset_dir = cpath.parent / "data"
results_path = cpath.parent / "results" / "vina"

RUN_DOCKING=False

In [4]:
datasets = [
    'bindingdb', # reference dataset should be the first
    'bindingdb_active',
    'protobind_diff',
    'pocket2mol',
    'pocketflow',
    'targetdiff',
    'reinvent',
    'tamgen',
]
annotation = pd.read_csv(cpath.parent / "paper" / 'tables' / 'selected_targets_benchmark.csv', index_col='Name')
gene2pdb = dict(annotation['PDB name'])
annotation

,Number of samples in the train set,Dataset type,Family name,UniProt ID,PDB name,L1 family name,L2 family name,sequence_int
Name,,,,,,,,
ESR1,4483,easy,Nuclear receptor,P03372,2r6w,Transcription factor,Nuclear receptor,138
HCRTR1,12691,easy,GPCR,O43613,4zjc,Membrane receptor,Family A G protein-coupled receptor,176
JAK1,12455,easy,Kinase,P23458,3eyg,Enzyme,Kinase,1145
P2RX3,5140,easy,Ion channel,P56373,5svl,Ion channel,Ligand-gated ion channel,2054
KDM1A,4622,easy,Protein-protein interaction target,O60341,5lhg,Epigenetic regulator,Eraser,2281
IDH1,5177,easy,Non-kinase enzyme,O75874,4umx,Enzyme,Oxidoreductase,2908
RIOK1,15,hard,Kinase,Q9BRS2,4otp,Enzyme,Kinase,1163
NR4A1,28,hard,Nuclear receptor,P22736,3v3q,Transcription factor,Nuclear receptor,1230
GRIK1,335,hard,Ion channel,P39086,3fv1,Ion channel,Ligand-gated ion channel,1852


In [ ]:
# DOCKER binary
CONFIG = 'laptop'

#### do not change these
dockstream_path = '' #path to dockstream
target_preparator = dockstream_path + "/target_preparator.py"
docker = dockstream_path + "/docker.py"
vina_binary_location = ""
# docking_config = output_dir / 'config' / 'vina_docking.json'


# DOKING PARAMS
num_poses = 3
number_cores = os.cpu_count()
exhaustiveness = 16
delta_X = 5
delta_Y = 5
delta_Z = 5
box_min_size=20

## Preprocess PDB and pockets

In [6]:
for gene, pdb in gene2pdb.items():
    if not RUN_DOCKING:
        continue
    pdb_protein_path = pdb_dir / pdb /  f"{pdb}_protein_cleaned.pdb"
    reference_ligand_path = pdb_dir / pdb /  f"{pdb}_ligand.sdf"
    assert pdb_protein_path.exists()
    assert reference_ligand_path.exists()

    
    # generate output paths for the configuration file, the "fixed" PDB file and the "rDock" cavity
    target_prep_path = output_dir / f"{gene}_prep.json"
    fixed_pdb_path = output_dir / f"{gene}_fixed_target.pdb"
    pdbqt_path = output_dir / f"{gene}.pdbqt"
    log_file_target_prep = output_dir / f"{gene}_target_prep.log"

    
    # specify the target preparation JSON file as a dictionary and write it out
    tp_dict = {
      "target_preparation":
      {
        "header": {                                   # general settings
          "logging": {                                # logging settings (e.g. which file to write to)
            "logfile": str(log_file_target_prep)
          }
        },
        "input_path": str(pdb_protein_path),          # this should be an absolute path
        "fixer": {                                    # based on "PDBFixer"; tries to fix common problems with PDB files
          "enabled": True,
          "standardize": True,                        # enables standardization of residues
          "remove_heterogens": True,                  # remove hetero-entries
          "fix_missing_heavy_atoms": True,            # if possible, fix missing heavy atoms
          "fix_missing_hydrogens": True,              # add hydrogens, which are usually not present in PDB files
          "fix_missing_loops": False,                 # add missing loops; CAUTION: the result is usually not sufficient
          "add_water_box": False,                     # if you want to put the receptor into a box of water molecules
          "fixed_pdb_path": str(fixed_pdb_path)            # if specified and not "None", the fixed PDB file will be stored here
        },
        "runs": [                                     # "runs" holds a list of backend runs; at least one is required
          {
            "backend": "AutoDockVina",                # one of the backends supported ("AutoDockVina", "OpenEye", ...)
            "output": {
              "receptor_path": str(pdbqt_path)        # the generated receptor file will be saved to this location
            },
            "parameters": {
              "pH": 7.4,                              # sets the protonation states (NOT used in Vina)
              "extract_box": {                        # in order to extract the coordinates of the pocket (see text)
                "reference_ligand_path":              # path to the reference ligand
                  str(reference_ligand_path),  
                "reference_ligand_format": "sdf"      # format of the reference ligand
              }
    }}]}}
    
    with open(target_prep_path, 'w') as f:
        json.dump(tp_dict, f, indent="    ")
        
    # execute this in a command-line environment after replacing the parameters
    !{sys.executable} {target_preparator} -conf {target_prep_path}
    !head -n 25 {log_file_target_prep}

## Start DOCKING

### Default config

In [7]:
default_config = {
  "docking": {
    "header": {
      "logging": {
        "logfile": "vina_docking.log"
      }
    },
    "ligand_preparation": {
      "embedding_pools": [
        {
          "pool_id": "RDkit_pool",
          "type": "RDkit",
          "parameters": {
            "removeHs": False,
            "coordinate_generation": {
              "method": "UFF",
              "maximum_iterations": 300
            }
          },
          "input": {
            "standardize_smiles": False,
            "type": "csv",
            "input_path": ".",
            "delimiter": ",",
            "columns": {
              "smiles": "SMILES",
              "names": "drug_id"
            }
          },
          "output": {
            "conformer_path": "<conformer_path>",
            "format": "sdf"
          }
        }
      ]
    },
    "docking_runs": [
      {
        "backend": "AutoDockVina",
        "run_id": "AutoDockVina",
        "input_pools": [
          "RDkit_pool"
        ],
        "parameters": {
          "binary_location": "<binary_path>",
          "parallelization": {
            "number_cores": 16
          },
          "seed": 42,
          "receptor_pdbqt_path": [
            "<receptor_path>"
          ],
          "number_poses": 5,
          "exhaustiveness": 16,
          "search_space": {
            "--center_x": None,
            "--center_y": None,
            "--center_z": None,
            "--size_x": None,
            "--size_y": None,
            "--size_z": None
          }
        },
        "output": {
          "poses": {
            "poses_path": "<poses_path>"
          },
          "scores": {
            "scores_path": "<scores_path>"
          }
        }
      }
    ]
  }
}

def get_box_params(filename):
    with open(filename, 'r') as f:
        text = f.read()

    # Regular expressions to match coordinate lines
    x_match = re.search(r"X coordinates: min=([\d\.\-]+), max=([\d\.\-]+)", text)
    y_match = re.search(r"Y coordinates: min=([\d\.\-]+), max=([\d\.\-]+)", text)
    z_match = re.search(r"Z coordinates: min=([\d\.\-]+), max=([\d\.\-]+)", text)

    xmin, xmax = float(x_match.group(1)), float(x_match.group(2))
    ymin, ymax = float(y_match.group(1)), float(y_match.group(2))
    zmin, zmax = float(z_match.group(1)), float(z_match.group(2))

    return  xmin, xmax, ymin, ymax, zmin, zmax

### Run dockig for each dataset and target

In [ ]:
for dataset in datasets:
    if not RUN_DOCKING:
        continue
    for gene, pdb in gene2pdb.items():
        
        output_dataset_dir = output_dir / dataset
        # Run test
        ligand_input_csv = dataset_dir / dataset / f"boltz_{gene}.csv"
        ligands_prep_path = output_dataset_dir / f"{gene}_prep_ligands.sdf"
        ligands_docked_path = output_dataset_dir / f"{gene}_docked_ligands.sdf"
        ligands_scores_path = output_dataset_dir / f"{gene}_scores.csv"
        config_tmp_dir = output_dataset_dir  / f"{gene}_vina_docking.json"
        ligands_docked_log = output_dataset_dir / f"{gene}_vina_docking.log"
        pdbqt_path = output_dir / f"{gene}.pdbqt"

        if not ligand_input_csv.exists():
            print(f"Input file file for {gene} not found in {output_dataset_dir}")
            continue

        if ligands_scores_path.exists():
            print(f"Scores for {gene} found in {output_dataset_dir}. Spipping")
            continue

        os.makedirs(output_dataset_dir, exist_ok=True)

        print(f"Running docking for {dataset}/{gene}")
        conf = copy.deepcopy(default_config)
        conf['docking']['docking_runs'][0]['output']['poses']['poses_path'] = str(ligands_docked_path)
        conf['docking']['docking_runs'][0]['output']['scores']['scores_path'] = str(ligands_scores_path)
        conf['docking']['header']['logging']['logfile'] = str(ligands_docked_log)
        conf['docking']['ligand_preparation']['embedding_pools'][0]['output']['conformer_path'] = str(ligands_prep_path)
        conf['docking']['docking_runs'][0]['parameters']['binary_location'] = str(vina_binary_location)
        conf['docking']['docking_runs'][0]['parameters']['receptor_pdbqt_path'] = [str(pdbqt_path)]

        # SETUP DOCKIG PARAMS
        conf['docking']['docking_runs'][0]['parameters']['parallelization']['number_cores'] = number_cores
        conf['docking']['docking_runs'][0]['number_poses'] = num_poses
        conf['docking']['docking_runs'][0]['exhaustiveness'] = exhaustiveness
        
        # SETUP BOX
        X_min, X_max, Y_min, Y_max, Z_min, Z_max = get_box_params(output_dir / f"{gene}_target_prep.log")
        conf['docking']['docking_runs'][0]['parameters']['search_space']['--center_x'] = (X_max + X_min) / 2
        conf['docking']['docking_runs'][0]['parameters']['search_space']['--center_y'] = (Y_max + Y_min) / 2
        conf['docking']['docking_runs'][0]['parameters']['search_space']['--center_z'] = (Z_max + Z_min) / 2
        conf['docking']['docking_runs'][0]['parameters']['search_space']['--size_x'] = max((X_max - X_min) + delta_X*2, box_min_size)
        conf['docking']['docking_runs'][0]['parameters']['search_space']['--size_y'] = max((Y_max - Y_min) + delta_Y*2, box_min_size)
        conf['docking']['docking_runs'][0]['parameters']['search_space']['--size_z'] = max((Z_max - Z_min) + delta_Z*2, box_min_size)
        
        # change smiles input
        smiles_input = conf['docking']['ligand_preparation']['embedding_pools'][0]['input']
        # smiles_input['type'] = 'sdf'
        smiles_input['input_path'] = str(ligand_input_csv)
        conf['docking']['ligand_preparation']['embedding_pools'][0]['input'] = smiles_input
        
        # save updated version in output_dir dir
        with open(config_tmp_dir, 'wt') as f:
            json.dump(conf,  f, indent=2)
        !{sys.executable} {docker} -conf {config_tmp_dir} -print_scores

        # copy scores to results folder
        os.makedirs(results_path / dataset, exist_ok=True)
        ligands_results_path = results_path / dataset / f"{gene}_scores.csv"
        if ligands_scores_path.exists():
            shutil.copy(ligands_scores_path, ligands_results_path)
        else:
            print(f"Missing Vina scores for {dataset} {gene}")
            

## Load scores

In [5]:
scores = {}
for dataset in datasets:
    output_dataset_dir = results_path / dataset
    all_scores = list(output_dataset_dir.glob('*_scores.csv'))
    if len(all_scores) == 0:
        print(f"No dockig results for {dataset}")
        continue
    else:
        print(f"Found {len(all_scores)} targets for {dataset}")

    df_ = pd.concat([pd.read_csv(path) for path in all_scores])
    df_ = df_[df_.lowest_conformer]
    df_['dataset'] = dataset
    df_ = df_.drop(columns=['ligand_number', 'enumeration', 'conformer_number', 'lowest_conformer'])
    df_['gene_id'] = [x.split('_')[0] for x in df_['name']]
    scores[dataset] = df_

Found 12 targets for bindingdb
Found 12 targets for bindingdb_active
Found 12 targets for protobind_diff
Found 12 targets for pocket2mol
Found 12 targets for pocketflow
Found 12 targets for targetdiff
Found 12 targets for reinvent
Found 12 targets for tamgen


In [6]:
for dataset, df_ in scores.items():
    print(dataset.upper())
    display(df_.groupby('gene_id')['score'].agg(['mean', 'sem', 'count']))

BINDINGDB


,mean,sem,count
gene_id,,,
CCR9,-6.194072,0.677425,97
ESR1,-7.913155,0.096475,97
FTO,-8.388021,0.101775,97
GRIK1,0.609660,0.974853,97
HCRTR1,-8.696247,0.523158,97
IDH1,-8.766031,0.102751,97
JAK1,-7.921918,0.423180,97
KDM1A,-8.407258,0.110929,97
NR4A1,-7.118165,0.079720,97


BINDINGDB_ACTIVE


,mean,sem,count
gene_id,,,
CCR9,-8.505871,0.244808,85
ESR1,-8.936520,0.113297,100
FTO,-8.085516,0.170518,31
GRIK1,-5.091691,0.301431,97
HCRTR1,-9.929280,0.073859,100
IDH1,-8.868953,0.102966,86
JAK1,-8.980592,0.098375,98
KDM1A,-8.460949,0.091584,99
NR4A1,-6.643400,0.236848,20


PROTOBIND_DIFF


...

,mean,sem,count
gene_id,,,
CCR9,-7.402900,0.280331,100
ESR1,-7.994561,0.108191,98
FTO,-7.350847,0.096630,98
GRIK1,2.124960,0.933148,99
HCRTR1,-8.301740,0.101299,100
IDH1,-8.670794,0.093408,97
JAK1,309439.718052,309447.153989,97
KDM1A,315626.309283,315633.549467,99
NR4A1,-6.141070,0.064004,100


REINVENT


,mean,sem,count
gene_id,,,
CCR9,-9.259340,0.088934,100
ESR1,-8.656490,0.081679,100
FTO,-7.795570,0.074416,100
GRIK1,-6.108071,0.118017,99
HCRTR1,-10.082460,0.075849,87
IDH1,-9.750770,0.055972,100
JAK1,-8.678620,0.080723,100
KDM1A,-8.202500,0.073571,100
NR4A1,-6.399660,0.056968,100


TAMGEN


,mean,sem,count
gene_id,,,
CCR9,-8.30861,0.126066,100
ESR1,-6.48076,0.084315,100
FTO,-6.80634,0.080060,100
GRIK1,-3.11875,0.294150,100
HCRTR1,-7.42286,0.106647,100
IDH1,-6.19107,0.091012,100
JAK1,-6.37932,0.081604,100
KDM1A,-7.56468,0.087049,100
NR4A1,-6.68033,0.075941,100
